### Importing statements


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, learning_curve
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer

import joblib
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout,Input, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import json
from sklearn.model_selection import RandomizedSearchCV
import scipy
import json
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input, Concatenate, LSTM


## Data Loading

Loaded the save data from Feature Engineering

In [2]:

# Load the CSV file into a DataFrame
data = pd.read_csv('/content/drive/MyDrive/Relevant_health_data.csv')

# Display the first few rows of the DataFrame
data.head()

,index,Timestamp,Subreddit,Type,Post_id,Title,Author,Text,Score,Total_comments,month,hour,day_of_week,Sentiment,Title_word_count,Text_word_count,Category,Processed_Title,Processed_Text
0,0,2018-05-28 19:41:32,mentalhealth,Comment,8msp9v,Mental Health Awareness Month: I have schizoaf...,Candle1nTh3Dark,"Thank you for posting this, I've failed univer...",141.0,0.0,5,19,Monday,negative,48,20,MH_Improvement,mental health awareness month schizoaffective ...,thank posting failed university 3 time due hos...
1,1,2018-05-28 19:53:47,mentalhealth,Comment,8msp9v,Mental Health Awareness Month: I have schizoaf...,sciencelove1994,I have schizoaffective mixed episode type. I a...,61.0,0.0,5,19,Monday,positive,48,33,MH_Improvement,mental health awareness month schizoaffective ...,schizoaffective mixed episode type junior ivy ...
2,2,2018-05-28 20:04:29,mentalhealth,Comment,8msp9v,Mental Health Awareness Month: I have schizoaf...,Unknown,As someone who is struggling with mental illne...,36.0,0.0,5,20,Monday,negative,48,49,MH_Improvement,mental health awareness month schizoaffective ...,someone struggling mental illness currently co...
3,3,2018-05-28 20:00:07,mentalhealth,Comment,8msp9v,Mental Health Awareness Month: I have schizoaf...,swild89,I’m heading back now at 28! Thanks for the ins...,28.0,0.0,5,20,Monday,positive,48,11,MH_Improvement,mental health awareness month schizoaffective ...,heading back 28 thanks inspiration
4,4,2018-05-28 21:21:11,mentalhealth,Comment,8msp9v,Mental Health Awareness Month: I have schizoaf...,BoozeMeUpScotty,Yayyyy!!! Good for you! This is my favorite ki...,11.0,0.0,5,21,Monday,positive,48,499,MH_Improvement,mental health awareness month schizoaffective ...,yayyyy good favorite kind post see anywhere wo...


> The above data has the preprocessed title, text and category

In [3]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838721 entries, 0 to 838720
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   index             838721 non-null  int64  
 1   Timestamp         838721 non-null  object 
 2   Subreddit         838721 non-null  object 
 3   Type              838721 non-null  object 
 4   Post_id           838721 non-null  object 
 5   Title             838721 non-null  object 
 6   Author            838721 non-null  object 
 7   Text              838721 non-null  object 
 8   Score             838721 non-null  float64
 9   Total_comments    838721 non-null  float64
 10  month             838721 non-null  int64  
 11  hour              838721 non-null  int64  
 12  day_of_week       838721 non-null  object 
 13  Sentiment         838721 non-null  object 
 14  Title_word_count  838721 non-null  int64  
 15  Text_word_count   838721 non-null  int64  
 16  Category          83

In [4]:
data.isnull().sum()

,0
index,0
Timestamp,0
Subreddit,0
Type,0
Post_id,0
Title,0
Author,0
Text,0
Score,0
Total_comments,0


In [5]:
data = data.dropna(subset=['Processed_Title', 'Processed_Text']).reset_index(drop=True)


In [6]:
data.isnull().sum()

,0
index,0
Timestamp,0
Subreddit,0
Type,0
Post_id,0
Title,0
Author,0
Text,0
Score,0
Total_comments,0


In [7]:
pip install transformers

In [8]:
from transformers import pipeline

# Load pipeline with fast tokenizer
sentiment_analyzer = pipeline("sentiment-analysis",
                              model="distilbert-base-uncased-finetuned-sst-2-english",
                              tokenizer="distilbert-base-uncased-finetuned-sst-2-english")



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [9]:
# Function to analyze sentiment
def analyze_sentiment(text):
    if pd.isnull(text):  # Handle null values
        return None, None
    try:
        # Analyze sentiment (truncated to 512 tokens to fit the model's input limit)
        result = sentiment_analyzer(text[:512])
        sentiment = result[0]['label']  # Positive/Negative
        score = result[0]['score']  # Confidence score
        # print(f"Sentiment: {sentiment}, Score: {score}")
        return sentiment, score
    except Exception as e:
        print(f"Error processing text: {e}")
        return None, None


In [10]:

# Split data into 5 parts
data_parts = np.array_split(data, 20)


In [ ]:

# Specify the already processed part index
processed_part_index = -1  # Update this to the index of the processed part

# Process only the remaining parts
file_paths = []
for i, part in enumerate(data_parts):
    if i != processed_part_index:
        print(f"Skipping already processed part {i+1}...")
        continue  # Skip the already processed part

    print(f"Processing part {i+1}...")
    part['Sentiment2'], part['Sentiment_Score'] = zip(*part['Processed_Text'].apply(analyze_sentiment))

    # Save each part to a CSV file
    file_path = f'/content/drive/MyDrive/new_health_data_part_{i+1}.csv'
    part.to_csv(file_path, index=False)
    file_paths.append(file_path)

# Add the already processed part to file_paths
file_paths.insert(processed_part_index, f'/content/drive/MyDrive/new_health_data_part_{processed_part_index+1}.csv')

# Merge all processed parts
processed_data = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

# Save the merged data
merged_file_path = '/content/drive/MyDrive/health_data_merged.csv'
processed_data.to_csv(merged_file_path, index=False)

print(f"Merged file saved at {merged_file_path}")

Skipping already processed part 1...
Processing part 2...
